# Sentinel-1 Level 0 Data Decoding Example

Sentinel-1 is a Synthetic Aperture Mapping satellite constellation operated by the European Space Agency (ESA). ESA publish several types of product associated with each data acquisition. Level 1 and Level 2 data files consist of various types of processed SAR images, but the raw packetized data downlinked to the ground is also available in the form of Level 0 products.

ESA also publish details of the [image formation algorithm used to generate Level 1 products](https://sentinels.copernicus.eu/web/sentinel/user-guides/document-library/-/asset_publisher/xlslt4309D5h/content/id/4629294?_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_xlslt4309D5h_redirect=https%3A%2F%2Fsentinels.copernicus.eu%2Fweb%2Fsentinel%2Fuser-guides%2Fdocument-library%3Fp_p_id%3Dcom_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_xlslt4309D5h%26p_p_lifecycle%3D0%26p_p_state%3Dnormal%26p_p_mode%3Dview%26_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_xlslt4309D5h_cur%3D2%26_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_xlslt4309D5h_delta%3D10%26p_r_p_resetCur%3Dfalse%26_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_xlslt4309D5h_assetEntryId%3D4629294) and [structure of Sentinel-1 data packets](https://sentinels.copernicus.eu/web/sentinel/user-guides/document-library/-/asset_publisher/xlslt4309D5h/content/id/3120468?_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_xlslt4309D5h_redirect=https%3A%2F%2Fsentinels.copernicus.eu%2Fweb%2Fsentinel%2Fuser-guides%2Fdocument-library%3Fp_p_id%3Dcom_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_xlslt4309D5h%26p_p_lifecycle%3D0%26p_p_state%3Dnormal%26p_p_mode%3Dview%26_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_xlslt4309D5h_cur%3D13%26_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_xlslt4309D5h_delta%3D10%26p_r_p_resetCur%3Dfalse%26_com_liferay_asset_publisher_web_portlet_AssetPublisherPortlet_INSTANCE_xlslt4309D5h_assetEntryId%3D3120468) in their [document library](https://sentinels.copernicus.eu/web/sentinel/user-guides/document-library).

This notebook demonstrates data extraction from Level 0 products using the Sentinel1Decoder Python code at https://github.com/Rich-Hall/sentinel1decoder. An example implementation of the range-Doppler algorithm is also provided to demonstrate image formation from this data.

## 1 - Imports and setup

In [ ]:
import sentinel1decoder
import pandas as pd
import numpy as np
import logging
import math
import cmath
import struct
import os as system
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.interpolate import interp1d

In [ ]:
filepath = ""
filename = "S1A_IW_RAW__0SDV_20240609T180612_20240609T180644_054250_06993F_39D8\s1a-iw-raw-s-vh-20240609t180612-20240609t180644-054250-06993f-annot.dat"
#filepath = "../data/russia/"
#filename = "s1a-iw-raw-s-vh-20230307t152137-20230307t152158-047540-05b562.dat"

inputfile = "S1A_S3_RAW__0SDH_20240617T213605.SAFE\S1A_S3_RAW__0SDH_20240617T213605_20240617T213630_054368_069D4F_ABA9.SAFE\s1a-s3-raw-s-hh-20240617t213605-20240617t213630-054368-069d4f.dat"
l0file = sentinel1decoder.Level0File(inputfile)

## 2 - Extract File Metadata

Sentinel-1 level 0 data files consist of the raw packetized data sent to the ground. One packet typically consists of the radar instrument output associated with one radar echo, so a single file typically consists of many thousands of packets. Packets may also consist of other types of data e.g. background noise measurements for instrument calibration.

We are working with data acquired in stripmap mode over Sao Paulo and the nearby port of Santos, Brazil. Stripmap data is mainly used to monitor small islands, so is relatively infrequently used. However, it is relatively simple compared to Interferometric Wide Swath mode, the main acquisiton mode used over land, and therefore makes our task of image formation much simpler!

Initially we're going to pull the metadata from each packet and output to a Pandas dataframe to examine the file contents. Producing an image from the entirity of the data found in a single file would take a long time and require a lot of memory, so we're aiming to produce an image from just a subset of this data.

In [ ]:
l0file.packet_metadata

The satellite ephemeris data is sub-commutated across multiple packets due to its relatively low update rate, so we need to perform an extra step to extract this information.

In [ ]:
l0file.ephemeris

## 3 - Extract Data

### 3.1 - Select Packets to Process

Now we've extracted all the packet metadata, we're going to select the data packets we'll be processing. We want to exclude all packets that don't contain SAR instrument returns, and then pick a small set of these to operate on. For this example we'll be focusing on the coastline around the port of Santos.

In [ ]:
selected_burst = 8
selection = l0file.get_burst_metadata(selected_burst)
selection

### 3.2 - Extract Raw I/Q Sensor Data

Now we're ready to extract the raw sensor output from the file. The result will be a set of complex I/Q samples measured by the SAR instrument. By stacking these horizontally we can produce a 2D array of data samples, with fast time $\tau$ along one axis and slow time $\eta$ along the other. Since all the required information to do this is contained in packet metadata, the decoder outputs data arranged like this automatically.

In [ ]:
# Decode the IQ data
original_radar_data = l0file.get_burst_data(selected_burst, True)

# Cache this data so we can retrieve it more quickly next time we want it
# l0file.save_burst_data(selected_burst)

In [ ]:
#l0file.save_burst_data(selected_burst)

Plotting our array, we can see that although there is clearly some structure to the data, we can't yet make out individual features. Our image needs to be focused along both the range and azimuth axes.

In [ ]:
# Truncate the data
slow_time_truncated = int(original_radar_data.shape[1] * 0.2)
radar_data = original_radar_data[:, :slow_time_truncated]

In [ ]:
SNR = 10**(5/10)

def generate_thermal_noise(shape, sigma):
    noise_real = np.random.normal(0, sigma, shape)
    noise_imag = np.random.normal(0, sigma, shape)
    return noise_real + 1j * noise_imag

mean_value = np.mean(abs(radar_data)**2)
additional_noise_power = mean_value / SNR
additional_noise = generate_thermal_noise(radar_data.shape, np.sqrt(additional_noise_power/2))

# Add noise to radar data
radar_data += additional_noise

# Plot the noisy IQ data
plt.subplot(2, 1, 2)
plt.title("Noisy Sentinel-1 Raw I/Q Sensor Output")
plt.imshow(abs(radar_data[:, :]), vmin=0, vmax=15, origin='lower', )
plt.xlabel("Fast Time (down range)")
plt.ylabel("Slow Time (cross range)")
plt.tight_layout()
plt.colorbar()
plt.show()

## 4 - Image Processing

The following section demonstrates an implementation of the range-Doppler algorithm. This essentially consists of the following steps:
- Range compression
- Transform to range-Doppler domain
- Range Cell Migration Correction (RCMC)
- Azimuth compression
- Transform to time domain
- Image formation

### 4.1 - Define auxiliary parameters

We require a number of parameters in the calculations that follow, so we'll define them all here. These are:
- Image sizes
- Various transmitted pulse parameters used to synthesize a replica Tx pulse
- Sample rates in range and azimuth
- The fast time $\tau$ associated with each range sample along a range line, and the corresponding slant range of closest approach $R_{0}$ for each of these range samples
- The frequency axes in range $f_{\tau}$ and azimuth $f_{\eta}$ after transforming our array to the frequency domain
- The effective spacecraft velocity $V_{r}$. This is a psuedo velocity approximated by $V_{r} \approx \sqrt{V_{s} V_{g}}$, where $V_{s}$ is the norm of the satellite velocity vector, and $V_{g}$ is the antenna beam velocity projected onto the ground. $V_{g}$ is calculated numerically acording to the method defined in https://iopscience.iop.org/article/10.1088/1757-899X/1172/1/012012/pdf. Note that $V_{g}$ and hence $V_{r}$ varies by slant range.
- The cosine of the instantaneous squint angle $D(f_{\eta}, V_{r})$, where

$$D(f_{\eta}, V_{r}) = \sqrt{1 - \frac{c^{2} f_{\eta}^{2}}{4 V_{r}^{2} f_{0}^{2}}}$$


In [ ]:
len_range_line = radar_data.shape[1]
len_az_line = radar_data.shape[0]


In [ ]:
print(len_range_line)
print(len_az_line)

# Tx pulse parameters
c = sentinel1decoder.constants.SPEED_OF_LIGHT_MPS
RGDEC = selection["Range Decimation"].unique()[0]
PRI = selection["PRI"].unique()[0]
rank = selection["Rank"].unique()[0]
suppressed_data_time = 320/(8*sentinel1decoder.constants.F_REF)
range_start_time = selection["SWST"].unique()[0] + suppressed_data_time
wavelength = sentinel1decoder.constants.TX_WAVELENGTH_M

# Sample rates
range_sample_freq = sentinel1decoder.utilities.range_dec_to_sample_rate(RGDEC)
range_sample_period = 1/range_sample_freq
az_sample_freq = 1 / PRI
az_sample_period = PRI

# Fast time vector - defines the time axis along the fast time direction
sample_num_along_range_line = np.arange(0, len_range_line, 1)
fast_time_vec = range_start_time + (range_sample_period * sample_num_along_range_line)

# Slant range vector - defines R0, the range of closest approach, for each range cell
slant_range_vec = ((rank * PRI) + fast_time_vec) * c/2
    
# Axes - defines the frequency axes in each direction after FFT
SWL = len_range_line/range_sample_freq
az_freq_vals = np.arange(-az_sample_freq/2, az_sample_freq/2, 1/(PRI*len_az_line))
range_freq_vals = np.arange(-range_sample_freq/2, range_sample_freq/2, 1/SWL)
 
# Spacecraft velocity - numerical calculation of the effective spacecraft velocity
ecef_vels = l0file.ephemeris.apply(lambda x: math.sqrt(x["X-axis velocity ECEF"]**2 + x["Y-axis velocity ECEF"]**2 +x["Z-axis velocity ECEF"]**2), axis=1)
velocity_interp = interp1d(l0file.ephemeris["POD Solution Data Timestamp"].unique(), ecef_vels.unique(), fill_value="extrapolate")
x_interp = interp1d(l0file.ephemeris["POD Solution Data Timestamp"].unique(), l0file.ephemeris["X-axis position ECEF"].unique(), fill_value="extrapolate")
y_interp = interp1d(l0file.ephemeris["POD Solution Data Timestamp"].unique(), l0file.ephemeris["Y-axis position ECEF"].unique(), fill_value="extrapolate")
z_interp = interp1d(l0file.ephemeris["POD Solution Data Timestamp"].unique(), l0file.ephemeris["Z-axis position ECEF"].unique(), fill_value="extrapolate")
space_velocities = selection.apply(lambda x: velocity_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)

x_positions = selection.apply(lambda x: x_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)
y_positions = selection.apply(lambda x: y_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)
z_positions = selection.apply(lambda x: z_interp(x["Coarse Time"] + x["Fine Time"]), axis=1).to_numpy().astype(float)

position_array = np.transpose(np.vstack((x_positions, y_positions, z_positions)))

a = sentinel1decoder.constants.WGS84_SEMI_MAJOR_AXIS_M
b = sentinel1decoder.constants.WGS84_SEMI_MINOR_AXIS_M
H = np.linalg.norm(position_array, axis=1)
W = np.divide(space_velocities, H)
lat = np.arctan(np.divide(position_array[:, 2], position_array[:, 0]))
local_earth_rad = np.sqrt(
    np.divide(
        (np.square(a**2 * np.cos(lat)) + np.square(b**2 * np.sin(lat))),
        (np.square(a * np.cos(lat)) + np.square(b * np.sin(lat)))
    )
)
cos_beta = (np.divide(np.square(local_earth_rad) + np.square(H) - np.square(slant_range_vec[:, np.newaxis]) , 2 * local_earth_rad * H))
ground_velocities = local_earth_rad * W * cos_beta

effective_velocities = np.sqrt(space_velocities * ground_velocities)

D = np.sqrt(
    1 - np.divide(
        wavelength**2 * np.square(az_freq_vals),
        4 * np.square(effective_velocities)
    )
).T

# We're only interested in keeping D, so free up some memory by deleting these large arrays.
del effective_velocities
del ground_velocities
del cos_beta
del local_earth_rad
del H
del W
del lat



### 4.2 - Convert data to 2D frequency domain

We're going to be doing almost all our calculations in the frequency domain, so the first step is to FFT along the azimuth and range axes.

In [ ]:
# FFT each range line
radar_data = np.fft.fft(radar_data, axis=1)
# FFT each azimuth line
radar_data = np.fft.fftshift(np.fft.fft(radar_data, axis=0), axes=0)

### 4.3 - Range compression - create and apply matched filter

Range compression is relatively simple. Range information is encoded in the arrival time of the pulse echo (i.e. an echo from a target further away will take longer to arrive), so by applying a matched filter consisting of the transmitted pulse, we can effectively focus the image along the range axis.

We can synthesize a replica of the Tx pulse from parameters specified in the packet metadata. Since we're operating in the frequency domain, we also need to transform our pulse replica that we're using as a matched filter to the frequency domain, then take the complex conjugate. FInally, we need to multiply every range line by our matched filter.

The Tx pulse replica is given by

$$\text{Tx Pulse} = exp\biggl\{2i\pi\Bigl(\bigl(\text{TXPSF} + \frac{\text{TXPRR  TXPL}}{2}\bigl)\tau + \frac{\text{TXPRR}}{2}\tau^{2}\Bigl)\biggl\}$$

where $\text{TXPSF}$ is the Tx Pulse Start Frequency, $\text{TXPRR}$ is the Tx Pulse Ramp Rate, and $\text{TXPL}$ is the Tx Pulse Length.

In [ ]:
# Create replica pulse
TXPSF = selection["Tx Pulse Start Frequency"].unique()[0]
TXPRR = selection["Tx Ramp Rate"].unique()[0]
TXPL = selection["Tx Pulse Length"].unique()[0]
num_tx_vals = int(TXPL*range_sample_freq)
tx_replica_time_vals = np.linspace(-TXPL/2, TXPL/2, num=num_tx_vals)
phi1 = TXPSF + TXPRR*TXPL/2
phi2 = TXPRR/2
tx_replica = np.exp(2j * np.pi * (phi1*tx_replica_time_vals + phi2*tx_replica_time_vals**2))

# Create range filter from replica pulse
range_filter = np.zeros(len_range_line, dtype=complex)

index_start = np.ceil((len_range_line - num_tx_vals) / 2)-1
index_end = num_tx_vals + np.ceil((len_range_line - num_tx_vals) / 2)-2

range_filter[int(index_start):int(index_end+1)] = tx_replica
range_filter = np.conjugate(np.fft.fft(range_filter))

# Apply filter
radar_data = np.multiply(radar_data, range_filter)

del range_filter
del tx_replica

### 4.4 - Range cell migration correction

Since the collector motion couples range and azimuth information, point targets will tend to produce returns spread in arcs across multiple range bins as the azimuth varies. We therefore need to apply a shift to align the phase history associated with each pointlike target into a single range bin, so we can then operate 1-dimensionally along the azimuth axis to perform azimuth compresison.

The RCMC shift is defined by

$$\text{RCMC shift} = R_{0} \biggl(\frac{1}{D} - 1\biggl)$$

with $D$ being the cosine of the instantaneous squint angle and $R_{0}$ the range of closest approach, both defined in section 4.1. Since we're still operating in the frequency domain we need to apply a filter of the form

$$\text{RCMC filter} = exp\biggl\{4i\pi\frac{f_{\tau}}{c}\bigl(\text{RCMC shift}\bigl)\biggl\}$$

Again, this needs to be multiplied by every range line in our data.

In [ ]:
# Create RCMC filter
range_freq_vals = np.linspace(-range_sample_freq/2, range_sample_freq/2, num=len_range_line)
rcmc_shift = slant_range_vec[0] * (np.divide(1, D) - 1)
rcmc_filter = np.exp(4j * np.pi * range_freq_vals * rcmc_shift / c)

# Apply filter
radar_data = np.multiply(radar_data, rcmc_filter)

del rcmc_shift
del rcmc_filter
del range_freq_vals

### 4.5 - Convert to range-Doppler domain

We've finished processing the image in range, so we can inverse FFT back to the range domain along the range axis. The image will still be in the frequency domain in azimuth.

In [ ]:
radar_data = np.fft.ifftshift(np.fft.ifft(radar_data, axis=1), axes=1)

### 4.6 - Azimuth compression - create and apply matched filter

Our azimuth filter is defined by

$$\text{Azimuth filter} = exp\biggl\{4i\pi\frac{R_{0}D(f_{\eta}, V_{r})}{\lambda}\biggl\}$$

In [ ]:
# Create filter
az_filter = np.exp(4j * np.pi * slant_range_vec * D / wavelength)

# Apply filter
radar_data = np.multiply(radar_data, az_filter)

del az_filter

### 4.7 - Transform back to range-azimuth domain

Finally, we'll transform back out of the frequency domain by taking the inverse FFT of each azimuth line.

In [ ]:
radar_data = np.fft.ifft(radar_data, axis=0)


## 5 - Plot Results - Import water binary map

With azimuth compression complete, we're ready to plot our image!

#### 5.1 - save binary map and scene image as npy arrays

In [ ]:
import rasterio

# Open the downloaded image with rasterio
with rasterio.open('Clipped_Image_box.tif') as src:
    water_array = src.read(1)  # read the first band
    meta_data = src.meta

np.save('water_body_map.npy', water_array)
np.save('scene_image.npy', abs(radar_data))

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

image_water_body = np.load('water_body_map.npy')  # Replace with your numpy array
image_scene = np.flipud(np.log10(np.load('scene_image.npy') + 1))

num_rows_to_remove = int(0.3 * image_water_body.shape[0])
image_water_body = image_water_body[:-num_rows_to_remove]

num_rows_to_remove = int(0.7 * image_water_body.shape[0])
image_water_body = image_water_body[num_rows_to_remove:]

num_rows_to_remove = int(0.3 * image_scene.shape[0])
image_scene = image_scene[:-num_rows_to_remove]

num_cols_to_remove = int(0.65 * image_water_body.shape[1])
image_water_body = image_water_body[:, :-num_cols_to_remove]

num_cols_to_remove = int(0.45 * image_water_body.shape[1])
image_water_body = image_water_body[:, num_cols_to_remove:]

# Define the downsampling factor (e.g., reduce by a factor of 2)
downsample_factor = 0.5
original_height, original_width = image_scene.shape[:2]
new_width = int(original_width * downsample_factor)
new_height = int(original_height * downsample_factor)
downsampled_image = cv2.resize(image_scene, (new_width, new_height), interpolation=cv2.INTER_AREA)
image_scene = cv2.resize(downsampled_image, (original_width, original_height), interpolation=cv2.INTER_NEAREST)

# Define the downsampling factor (e.g., reduce by a factor of 2)
downsample_factor = 0.5
original_height, original_width = image_water_body.shape[:2]
new_width = int(original_width * downsample_factor)
new_height = int(original_height * downsample_factor)
downsampled_image = cv2.resize(image_water_body, (new_width, new_height), interpolation=cv2.INTER_AREA)
image_water_body = cv2.resize(downsampled_image, (original_width, original_height), interpolation=cv2.INTER_NEAREST)

# Save or display the result
# cv2.imwrite('output_image.png', image_scene)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('image')
plt.imshow(image_scene, cmap='viridis')

image_water_body = np.where(image_water_body==0, 1, image_water_body)
plt.subplot(1, 2, 2)
plt.title('water_map')
plt.imshow(image_water_body, cmap='viridis')
plt.show()

def normalise(input):
    return (input - input.min()) / (input.max() - input.min())

def otsu_thresholding(image):
    image_8bit = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    threshold_value, binary_image = cv2.threshold(image_8bit, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary_image, threshold_value

# Perform otsu thresholding
S1_CLIPP = 0.5  # Clip threshold
image_scene = normalise(image_scene)
image_scene_clipped = normalise(np.clip(image_scene, 0, S1_CLIPP))

binary_diff, threshold_value = otsu_thresholding(image_scene_clipped)

#binary_diff = np.where(binary_diff > threshold_value, 1, 0)
binary_diff = np.where(binary_diff==0, 1, 0)
print(threshold_value)

if binary_diff.dtype != np.uint8:
    binary_diff = (binary_diff > 0).astype(np.uint8) * 255

num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_diff, connectivity=8)

min_size = 100000  # You can change this value according to your requirement
filtered_map = np.zeros_like(binary_diff)

for label in range(1, num_labels):  # Start from 1 to skip the background component
    if stats[label, cv2.CC_STAT_AREA] >= min_size:
        # If the component's area is greater than or equal to min_size, keep it
        filtered_map[labels == label] = 1

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('binary_diff')
plt.imshow(binary_diff, cmap='viridis')

plt.subplot(1, 2, 2)
plt.title('filtered_map')
plt.imshow(filtered_map, cmap='viridis')
plt.show()


np.save('filtered_map.npy', filtered_map)
np.save('image_water_body.npy', image_water_body)

In [ ]:
import os
import json

image_water_body = np.load('image_water_body.npy')
filtered_map = np.load('filtered_map.npy')

def adjust_brightness(img):
    norm_img = cv2.normalize(img, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    norm_img = np.uint8(norm_img)
    return norm_img

def resize_image(img, max_width=800, max_height=800):
    height, width = img.shape[:2]
    scaling_factor = min(max_width / width, max_height / height)
    new_width = int(width * scaling_factor)
    new_height = int(height * scaling_factor)
    resized_img = cv2.resize(img, (new_width, new_height))
    return resized_img, scaling_factor

def select_points(img, max_width=800, max_height=800):
    bright_img = adjust_brightness(img)
    resized_img, scaling_factor = resize_image(bright_img, max_width, max_height)
    clone = resized_img.copy()
    points = []

    def mouse_callback(event, x, y, flags, params):
        if event == cv2.EVENT_LBUTTONDOWN:
            points.append((x, y))
            cv2.circle(clone, (x, y), 5, (0, 255, 0), -1)
            cv2.imshow("Image", clone)

    cv2.imshow("Image", clone)
    cv2.setMouseCallback("Image", mouse_callback)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    # Scale points back to original image size
    original_points = [(int(x / scaling_factor), int(y / scaling_factor)) for (x, y) in points]
    return original_points

def align_images_manual(img1, img2, points1, points2, background_value=0, matrix_file='affine_matrix.json'):
    if os.path.exists(matrix_file):
        with open(matrix_file, 'r') as f:
            matrix = np.array(json.load(f))
    else:
        points1 = np.array(points1, dtype=np.float32)
        points2 = np.array(points2, dtype=np.float32)

        # Compute affine transformation matrix using more points
        matrix, _ = cv2.estimateAffinePartial2D(points2, points1)
        
        # Save the transformation matrix
        with open(matrix_file, 'w') as f:
            json.dump(matrix.tolist(), f)

    # Apply affine transformation to the second image
    aligned_img = cv2.warpAffine(img2, matrix, (img1.shape[1], img1.shape[0]), borderMode=cv2.BORDER_CONSTANT, borderValue=background_value)
    
    # Apply affine transformation to the mask
    mask = np.ones_like(img2, dtype=np.uint8) * 255
    aligned_mask = cv2.warpAffine(mask, matrix, (img1.shape[1], img1.shape[0]), borderMode=cv2.BORDER_CONSTANT, borderValue=0)

    # Combine the aligned image with the first image
    combined_image = np.where(aligned_mask == 255, aligned_img * 2, background_value)

    return combined_image

# Load images
img1 = image_water_body
img2 = filtered_map

# Ensure images are loaded properly
if img1 is None or img2 is None:
    print("Error loading images")
    exit()

# Select points manually on both images
print("Select 3 points on the first image")
points1 = select_points(img1.copy())

print("Select the corresponding 3 points on the second image")
points2 = select_points(img2.copy())

# Align images
background_value = 0.5  # Background value is 0
aligned_image = align_images_manual(img1, img2, points1, points2, background_value)

# Display the aligned image
plt.figure(figsize=(15, 5))
plt.subplot(1, 4, 1)
plt.title('Image 1')
plt.imshow(img1)
plt.axis('off')
plt.subplot(1, 4, 2)
plt.title('Image 2')
plt.imshow(img2)
plt.axis('off')
plt.subplot(1, 4, 3)
plt.title('Aligned Image 2')
plt.imshow(aligned_image)
plt.axis('off')
plt.subplot(1, 4, 4)
plt.title('Aligned Image - Map')
plt.imshow(aligned_image - img1)
plt.axis('off')
plt.show()

There are still a few noteworthy issues with our image. The first is folding - notice various terrain features from the top of the image are folded into the bottom, and terrain from the left of the image is folded into the right side. Folding in range (range ambiguities) occurs due to echoes spilling over into earlier or later sampling windows. Folding in azimuth occurs due to our sampling the azimuth spectrum of the scene at the PRF, which leads to folding in the frequency spectrum.

Various terrain features are clearly visible, however the image is still not perfectly focused. We have assumed a Doppler centroid of 0Hz, and have not applied a number of additional processing steps that ESA use to produce Level 1 products e.g. Secondary Range Compression (SRC). These are left as an exercise for the reader.

### Old Noise power calculator

- Trying to use absolute power values instead of SNR

In [ ]:
# def db_to_linear(value_db):
#     return 10 ** (value_db / 10)

# def calculate_reference_power(P_t, G_t_db, G_r_db, wavelength, sigma, R, L):
#     # Convert gains from dB to linear scale
#     G_t = db_to_linear(G_t_db)
#     G_r = db_to_linear(G_r_db)    
#     P_r = (P_t * G_t * G_r * (wavelength ** 2) * sigma) / ((4 * math.pi) ** 3 * R ** 4 * L)
#     return P_r

# def calculate_additional_noise_power_from_nesz(original_nesz_db, desired_nesz_db, ref_power):
#     # Convert NESZ from dB to linear scale
#     original_nesz_linear = db_to_linear(original_nesz_db)
#     desired_nesz_linear = db_to_linear(desired_nesz_db)
    
#     # Calculate the additional noise power needed
#     additional_noise_power = ref_power * (desired_nesz_linear - original_nesz_linear)
#     return additional_noise_power

# # Sentinel-1 Specifications
# P_t = 280  # Transmitted power in watts
# G_t_db = 80  # Transmitting antenna gain in dB
# G_r_db = 80  # Receiving antenna gain in dB
# wavelength = 0.05551712  # Wavelength in meters (5.4 GHz frequency)
# sigma = 1  # RCS in square meters
# R = 200e3  # Range in meters
# L_sentinel = db_to_linear(3)  # System losses (linear scale)
# sentinel_NESZ = -22

# sentinel_expected_power = calculate_reference_power(P_t, G_t_db, G_r_db, wavelength, sigma, R, L_sentinel)

# mean_value = np.mean(abs(radar_data)**2)

# print("Mean image value:", mean_value)
# print("Sentinel reference power:", sentinel_expected_power)

# # Capella Satellite Specifications
# Pt_capella = 10  # Transmitted power in watts
# gain_db_capella = 46
# G_capella = db_to_linear(gain_db_capella)  # Antenna gain (linear scale)
# wavelength_capella = 0.03189281  # Wavelength in meters (9.4 GHz frequency)
# R_capella = 200e3  # Range to the target in meters
# L_capella = db_to_linear(5)  # System losses (linear scale)
# capella_NESZ = -17

# ref_power = calculate_reference_power(Pt_capella, gain_db_capella, gain_db_capella, wavelength_capella, 1, R_capella, L_capella)
# print("Capella reference power:", ref_power)

# # Convert NESZ values to linear scale
# nesz_original_db = -22
# nesz_new_db = -17
# nesz_original_linear = 10**(nesz_original_db / 10)
# nesz_new_linear = 10**(nesz_new_db / 10)

# # Calculate the new noise power
# additional_noise_power = nesz_new_linear*mean_value
# #additional_noise_power = calculate_additional_noise_power_from_nesz(sentinel_NESZ, capella_NESZ, sentinel_expected_power)

# print("Additional noise power pre-scaling:", additional_noise_power)

# # Generate complex Gaussian noise
# def generate_thermal_noise(shape, sigma):
#     noise_real = np.random.normal(0, sigma, shape)
#     noise_imag = np.random.normal(0, sigma, shape)
#     return noise_real + 1j * noise_imag